In [1]:
import LRC_TME8_definitions_Allen as defs
import copy

## Exercice 1 : Implémentation de la composition

In [2]:
def transposeSet(S):
    return set([defs.transpose[r] for r in S])

In [3]:
transposeSet(['<','e','et','s'])

{'>', 'e', 'et', 'st'}

In [4]:
def symetrieSet(S):
    return set([defs.symetrie[r] for r in S])

In [5]:
symetrieSet(['<','e','et','s'])

{'>', 'e', 's', 'st'}

In [6]:
def compose(r1,r2):
    if r1 == '=':
        return {r2}
    if r2 == '=':
        return {r1}
    if (r1,r2) in defs.compositionBase:
        return defs.compositionBase[(r1,r2)]
    r1t = defs.transpose[r1]
    r2t = defs.transpose[r2]
    if (r2t,r1t) in defs.compositionBase:
        return transposeSet(defs.compositionBase[(r2t,r1t)])
    r1s = defs.symetrie[r1]
    r2s = defs.symetrie[r2]
    if (r1s,r2s) in defs.compositionBase:
        return symetrieSet(defs.compositionBase[(r1s,r2s)])
    r1st = defs.transpose[defs.symetrie[r1]]
    r2st = defs.transpose[defs.symetrie[r2]]
    if (r2st,r1st) in defs.compositionBase:
        return symetrieSet(transposeSet(defs.compositionBase[(r2st,r1st)]))

In [7]:
compose('=','d')

{'d'}

In [8]:
compose('m','d')

{'d', 'o', 's'}

In [9]:
compose('ot','>')

{'>'}

In [10]:
compose('>','e')

{'>'}

In [11]:
compose('ot','m')

{'dt', 'et', 'o'}

In [12]:
def compositionSet(S1, S2):
    S = set()
    for r1 in S1:
        for r2 in S2:
            S = S | compose(r1,r2)
    return S

In [13]:
compositionSet({'m','o'}, {'dt','et','st','='})

{'<', 'dt', 'et', 'm', 'o'}

## Exercice 2 : Gestion du graphe temporel

In [14]:
class Graphe:

    def __init__(self, noeuds, relations):
        self.noeuds = noeuds
        self.relations = relations
        
    def getRelations(self, i, j):
        if (i,j) in self.relations:
            return self.relations[(i,j)]
        if (j,i) in self.relations:
            return transposeSet(self.relations[(j,i)])
        return set(defs.transpose.keys())

In [15]:
g = Graphe({'A','B','C'},{('A','B') :{'o','e'}})

In [16]:
g.getRelations('A','B')

{'e', 'o'}

In [17]:
g.getRelations('A','C')

{'<', '=', '>', 'd', 'dt', 'e', 'et', 'm', 'mt', 'o', 'ot', 's', 'st'}

In [18]:
def propagation(g, a, b):
    Rab = g.getRelations(a,b)
    pile = [(a,b,Rab)]
    while pile:
        i,j,Rij = pile.pop()
        for k in g.noeuds:
            if k != i and k != j:
                Rik = g.getRelations(i,k)
                Rjk = g.getRelations(j,k)
                Rkj = g.getRelations(k,j)
                Rki = g.getRelations(k,i)
                newRik = Rik & compositionSet(Rij,Rjk)
                newRkj = Rkj & compositionSet(Rki,Rij)
                if not newRik or not newRkj:
                    break
                elif newRik != Rik:
                    Rik = newRik
                    g.relations[(i,k)] = Rik
                    pile.append((i,k,Rik))
                elif newRkj != Rkj:
                    Rkj = newRkj
                    g.relations[(k,j)] = Rkj
                    pile.append((k,j,Rkj))

In [19]:
def ajouter(g, R):
    G = copy.deepcopy(g)
    Rab, a, b = R
    if a not in g.noeuds:
        G.noeuds.add(a)
    if b not in g.noeuds:
        G.noeuds.add(b)
    G.relations[(a,b)] = Rab
    propagation(G,a,b)
    return G

Graphe g1 avec les relations A{<}B et A{>}C :

In [20]:
g1 = Graphe(set(),{})
g1 = ajouter(g1,({'<'},'A','B'))
g1.relations

{('A', 'B'): {'<'}}

In [21]:
g1 = ajouter(g1,({'>'},'A','C'))
g1.relations

{('A', 'B'): {'<'}, ('A', 'C'): {'>'}, ('B', 'C'): {'>'}}

Ajout de la relation B{=}C :

In [22]:
g1 = ajouter(g1,({'='},'B','C'))
g1.relations

{('A', 'B'): {'<'}, ('A', 'C'): {'>'}, ('B', 'C'): {'='}}

Graphe g2 avec les relations A{<}B et A{<}C :

In [23]:
g2 = Graphe(set(),{})
g2 = ajouter(g2,({'<'},'A','B'))
g2.relations

{('A', 'B'): {'<'}}

In [24]:
g2 = ajouter(g2,({'<'},'A','C'))
g2.relations

{('A', 'B'): {'<'}, ('A', 'C'): {'<'}}

Ajout de la relation B{=}C :

In [25]:
g2 = ajouter(g2,({'='},'B','C'))
g2.relations

{('A', 'B'): {'<'}, ('A', 'C'): {'<'}, ('B', 'C'): {'='}}

Graphe G1 établi à partir de la phrase "La  lampe  s’est  allumée  pendant  ou  juste  après  que  j’ai  appuyé  sur  l’interrupteur. John n’ ́etait pas dans la pièce lorsque j’ai appuye sur l’interrupteur" :

In [26]:
G1 = Graphe(set(),{})
G1 = ajouter(G1,({'ot','mt'},'L','S'))
G1 = ajouter(G1,({'>','<','m','mt'},'S','R'))
G1.relations

{('L', 'R'): {'<', '=', '>', 'dt', 'et', 'm', 'o', 's', 'st'},
 ('L', 'S'): {'mt', 'ot'},
 ('S', 'R'): {'<', '>', 'm', 'mt'}}

On apprend que John ́etait dans la pièce lorsque la lumière a ́été ́éteinte. On obtient alors le nouveau graphe G1 suivant :

In [31]:
G1 = ajouter(G1,({'o','s'},'L','R'))
G1.relations

{('L', 'R'): {'o', 's'}, ('L', 'S'): {'mt', 'ot'}, ('S', 'R'): {'<', 'm'}}

Cette dernière information nous permet donc de déduire que la lumière était allumée pendant une partie du temps où John était dans la pièce (soit elle a été allumée à l'arrivée de John et a été éteinte alors qu'il était encore dans la pièce (s) soit elle a été allumée avant l'arrivée de John et a été éteinte alors qu'il était encore dans la pièce (o)).

Graphe G2 de l'exercice sur le petit-déjeuner d'Alfred :

In [36]:
G2 = Graphe(set(),{})
G2 = ajouter(G2,({'o','et','dt','s','=','d','e','ot'},'J','D'))
G2 = ajouter(G2,({'et','=','e'},'J','C'))
G2 = ajouter(G2,({'<','m'},'D','P'))
G2.relations

{('D', 'C'): {'<', '=', 'd', 'dt', 'e', 'et', 'm', 'o', 'ot', 's', 'st'},
 ('D', 'P'): {'<', 'm'},
 ('J', 'C'): {'=', 'e', 'et'},
 ('J', 'D'): {'=', 'd', 'dt', 'e', 'et', 'o', 'ot', 's'},
 ('J', 'P'): {'<', 'dt', 'et', 'm', 'o'}}

On ajoute l'information suivante : "Boire le café est une étape du petit déjeuner" : 

In [37]:
G2 = ajouter(G2,({'s','=','d','e'},'C','D'))
G2.relations

{('C', 'D'): {'=', 'd', 'e', 's'},
 ('C', 'P'): {'<', 'm'},
 ('D', 'C'): {'<', '=', 'd', 'dt', 'e', 'et', 'm', 'o', 'ot', 's', 'st'},
 ('D', 'P'): {'<', 'm'},
 ('J', 'C'): {'=', 'e', 'et'},
 ('J', 'D'): {'=', 'd', 'e', 'et', 'o', 's'},
 ('J', 'P'): {'<', 'm'}}